# Validaciones con Pydantic

## Creacion de esquemas

Crearemos un esquema de datos que contenga toda la informacion relacionada con una pelicula. 

```py
from pydantic import BaseModel
...
...
class Movie(BaseModel):
    id: int | None = None
    title: str 
    year: int
    rating: float
    category: str
```
Ya tenemos los atributos de la clase Movie, la cual hereda de BaseModel, y contiene todos los atributos que contiene una pelicula. 

### Que significa: id: int | None = None 

*id: int | None = None* es una forma de declarar un parámetro en una función o en una clase:

- id: Esto es el nombre del parámetro. En este caso, se está declarando un parámetro llamado id.

- *int | None*: Esto es una anotación de tipo. En este caso, significa que el parámetro id puede ser de tipo **int** (número entero) o puede ser **None** (es decir, no tener valor). El operador | denota una unión de tipos, lo que indica que el parámetro puede ser de cualquiera de los tipos mencionados.

- *= None*: Esto es un valor por defecto. En este caso, el valor por defecto para id es None. Esto significa que si no se proporciona un valor para id al llamar a la función, se asumirá que su valor es **None**.

Se puede remplazar por:

```py
id: Optional[int] = None
```

Siempre y cuando se importe:

```py
from typing import Optional
```

### Modificando POST

Ojo tiene un bug, que ahora vamos a corregir 🐞

```py
@app.post('/movies', tags=['movies'])
def create_movie(movie: Movie):  👈  
    movies.append(movie)

    return movie
```
- Quiere decir que va requerir una *movie* de tipo *Movie*

#### Corrigiendo el bug.

Al momento de ir al navegador y hacer una peticion, para consultar la pelicula que se acabo de crear:

    http://127.0.0.1:5000/movies/101

Mostraba el siguiente error:

    if item['id'] == id:
    TypeError: 'Movie' object is not subscriptable

Esto es por el tipo de datos como se esta guardando, pues lo esta guardando como tipo *main.Movie*, asi que antes de guardarlo, hay que convertirlo a diccionario.

```py
@app.post('/movies', tags=['movies'])
def create_movie(movie: Movie):    
    movies.append(dict(movie)) 👈

    return movie
```


### Modificando PUT

```py
@app.put('/movies/{id}', tags=['movies'])
def update_movie(id: int, movie: Movie):
    
    for movie in movies:
        if movie['id'] == id:
            
            movie['title'] = movie.title
            movie['year'] = movie.year
            movie['rating'] = movie.rating
            movie['category'] = movie.category
            break
    
    return movie
```

### Verificando Esquema

![](https://i.imgur.com/wS1AJo0.png)

## Validaciones de tipos de datos

Hay cierto tipo de validaciones que ya estan integradas con FastAPI. Por ejemplo vamos a usar el metodo POST y en el body no enviaremos cualquier parametro, por ejemplo el *rating*:

{
  "id": 0,
  "title": "string",
  "year": 0,
  "category": "string"
}

Nos retornara un error *422 Unprocessable Entity*. 

![](https://i.imgur.com/HS108LI.png)

O por ejemplo, al realizar otra peticion con el metodo GET y en ves de enviar un *integer*, enviarmos un texto:

    http://127.0.0.1:5000/movies/a

Y mostrara:

{"detail":[{"type":"int_parsing","loc":["path","id"],"msg":"Input should be a valid integer, unable to parse string as an integer","input":"a","url":"https://errors.pydantic.dev/2.4/v/int_parsing"}]}

### Usando la clase Field de pydantic

    from pydantic import BaseModel, Field

#### Modificando la clase Movie

```py
class Movie(BaseModel):
    id: int | None = None
#    id: Optional[int] = None
    title: str = Field(default="movie title", min_length=5, max_length=15)
    year: int = Field(default=2022, le=2022)
    rating: float = Field(ge=0, le=10)
    category: str = Field(default="movie category")
```

- El titulo tenga un minimo de 5 caracteres y un maximo de 15.
- El titulo tenga un valor por defecto
- El año menor o igual al 2022, y por defecto 2022

![](https://i.imgur.com/feKe2st.png)

Tambien es posible dar los valores por defecto, creando una nueva clase *Config* dentro del BaseModel, y dentro de dicha clase:

```py
    class Config:
        schema_extra = {
            "example": {
                "id": 1,
                "title": "Mi película",
                "overview": "Descripción de la película",
                "year": 2022,
                "rating": 9.8,
                "category" : "Acción"
            }
        }

```

Aunque a mi no me funciono. 

## Validaciones de parametros

### Parametros de ruta

Se importa la clase *Path*:

    from fastapi import FastAPI, Body, Path

Y se modifica, para que el parametro de ruta tenga un valor entre 0 y 200:

    def get_movie(id: int = Path(ge=0, le=200)):


Error an intentar colocar 2001 en la 

{"detail":[{"type":"less_than_equal","loc":["path","id"],"msg":"Input should be less than or equal to 200","input":"2001","ctx":{"le":200},"url":"https://errors.pydantic.dev/2.4/v/less_than_equal"}]}

### Parametros Query

Se importa la clase Query:

    from fastapi import FastAPI, Body, Path, Query

Para que el parametro *category* sea minimo de 5 caracteres

    def get_movies_by_category(category: str = Query(min_length=5)):

## Tipos de respuestas

Ya trabajamos con HYMLResponse, pero hay otros tipos de respuesta, comenzaremso por importar la clase JSONResponse

    from fastapi.responses import HTMLResponse, JSONResponse

Servira para enviar respuestas en formato Json hacia el cliente. 

### Metodos GET

No tiene misterio, simplemente implementar el JSONResponse en la clase que retorna:

```py
@app.get('/movies', tags=['movies'])
def get_movies():
    return JSONResponse(content=movies) 👈
```

### Metodo POST

### Metodo PUT

```py
@app.put('/movies/{id}', tags=['movies'], response_model=Movie)
def update_movie(id: int, movie: Movie):
    
    for item in movies:
        if item['id'] == id:            
            item['title'] = movie.title
            item['year'] = movie.year
            item['rating'] = movie.rating
            item['category'] = movie.category
            break
    
    return JSONResponse(content={"message": "La película ha sido modificada", "modified_id": item["id"]})👈
```
 Aqui modifique el codigo para que retorara el *id* de la pelicula que se modifico

 #### Response Body

{
  "message": "La película ha sido modificada",
  "modified_id": 23
}

Como vez el contenido de la respuesta es un diccionario, porque el parametro *message* es un diccionario

### response_model

Especificar el response_model en FastAPI es importante por varias razones:

- Validación de Respuesta: El *response_model* permite a FastAPI validar que la respuesta que se está enviando desde la ruta coincide con el tipo de dato que se espera. Esto ayuda a evitar posibles errores de lógica o de tipo que puedan surgir durante el desarrollo.

- Documentación Automática: FastAPI utiliza la información del *response_model* para generar documentación automática sobre la API. Esto incluye la especificación de OpenAPI (anteriormente conocida como Swagger). Al especificar el tipo de dato que se espera en la respuesta, FastAPI puede proporcionar información detallada sobre la estructura de la respuesta, lo que facilita a los desarrolladores entender qué esperar de la API.

- Integración con Editores de Código: Si estás utilizando un editor de código o IDE que soporta herramientas de sugerencia de tipo (como autocompletado), especificar el *response_model* ayuda al editor a ofrecer sugerencias precisas sobre cómo interactuar con la respuesta de la API.

- Claridad en el Código: Es una buena práctica incluir el *response_model* porque proporciona una indicación clara de qué tipo de dato se espera que la función devuelva. Esto hace que el código sea más legible y fácil de entender para otros desarrolladores (o para ti mismo en el futuro).

#### Implementando Response_model en get_movies

Importar desde *typing* la clase *List*:

    from typing import Optional, List

Y ahora en la funcion:

```py
@app.get('/movies', tags=['movies'], response_model=List[Movie])
def get_movies() -> List[Movie]:
    return JSONResponse(content=movies)
```
- En el decorador, indica retornara una lista de objetos de tipo *Movie*
- Tambien puedo indicarlo al momento de declarar la funcion: *def get_movies() -> List[Movie]*

#### implementando response_model en get_movie

De manera muy similar:

```py
@app.get('/movies/{id}', tags=['movies'], response_model=Movie) 👈
def get_movie(id: int = Path(ge=0, le=200)) -> Movie: 👈
    for item in movies:
        if item['id'] == id:
            print(id)
            return JSONResponse(content=item)
        
    return JSONResponse(content=None)
```



### Para otros metodos 

Retorna un diccionario

```py
@app.post('/movies', tags=['movies'], response_model=dict) 👈
def create_movie(movie: Movie) -> dict:    👈
    movies.append(dict(movie))

    return JSONResponse(content={"message": "se ha creado la pelicula"})
```

#### Verificando su utilidad

Tomemos por ejemplo la funcion *delete_movie*, y veamos antes y despues de implementar *response model*, directamente en la documentacion:

##### Antes

![](https://i.imgur.com/VRoFX6p.png)

##### Despues

![](https://i.imgur.com/aPswoao.png)

## Códigos de estado

Distintos codigos de estados podemso usar:

https://developer.mozilla.org/en-US/docs/Web/HTTP/Status

Añadimos un nuevo parametro al decorador, con el codigo de estado que se va a ejecutar si la peticion se ejecuta correctamente

En realidad todas las rutas estan devolviendo el codigo de estado que deben devolver, a excepcion de POST:

```py
@app.post('/movies', tags=['movies'], response_model=dict, status_code=201) 👈
def create_movie(movie: Movie) -> dict:    
    movies.append(dict(movie))

    return JSONResponse(status_code=201, content={"message": "se ha creado la pelicula"}) 👈
```

Fijate la implementamso tanto en el decorador , como en el JSONResponse, aunque aparentente funcion bien implementadola solo en el el decorador.

### Implementando el codigo de estado 404.

The server cannot find the requested resource. In the browser, this means the URL is not recognized. In an API, this can also mean that the endpoint is valid but the resource itself does not exist. Servers may also send this response instead of 403 Forbidden to hide the existence of a resource from an unauthorized client. This response code is probably the most well known due to its frequent occurrence on the web.

404 = Not Found

#### La ruta es valida pero el recurso en si mismo no existe

Por ejemplo, cuando buscamos por el parametro de ruta, y la pelicula no existe. Actualmente solo estamos devolviendo None. Pero modifiquemola:

```py
@app.get('/movies/{id}', tags=['movies'], response_model=Movie, status_code=200)
def get_movie(id: int = Path(ge=0, le=200)) -> Movie:
    for item in movies:
        if item['id'] == id:
            print(id)
            return JSONResponse(content=item, status_code=200)
        
    return JSONResponse(content={"message": "Movie Not Found"}, status_code=404) 👈
```
